In [1]:
import sys
sys.path.append("src")
%load_ext autoreload
%autoreload 2
import pandas as pd
from scraping import scan_swedish_legislation_parallel
from vector_db import VectorDatabaseWeaviate
from embedd import split_text, EmbedderOpenAI, EmbedderSwedishLegislation
from search import SwedishLegislationSearch

## Scrap Swedish government office for all laws and regulations

In [2]:
df = scan_swedish_legislation_parallel()
df.to_pickle("data/laws.pickle")

100%|███████████████████████████████████████| 4539/4539 [01:42<00:00, 44.37it/s]


In [3]:
df = pd.read_pickle("data/laws.pickle")

## Set up embedding

In [2]:
if 0: # For debuging.. 
    df = pd.read_pickle("data/laws.pickle").sample(2)

df = pd.read_pickle("data/laws.pickle")

embedder = EmbedderSwedishLegislation(
    df=df,
    embedd_column="content",
    text_max_length=1500,
    text_overlap=150,
    context_columns=["title"],
)
embedder.setup()

In [ ]:
# Calculate the cost for the embedding
(embedder._df.n_tokens.sum() / 1000) * 0.0004

In [74]:
# Run the embedder
embedder.embedd(1000, timeout=1.01)
embedder._df.to_pickle("data/processed.pickle")

9791it [3:43:23,  1.37s/it]


In [3]:
embedder._df = pd.read_pickle("data/processed.pickle")

## Set up vector database

In [5]:
class_obj = VectorDatabaseWeaviate.setup_class_object_structure(
    class_name="Swedish_legislation",
    names=[
        "SFS_number", "title", "issuer", "content"
    ],
    data_types=["text"] * 6,
    description="Holds Swedish legislation"
)

vector_db = VectorDatabaseWeaviate(
    name="Swedish_legislation",
    url="http://weaviate:8080",
    schema=class_obj,
)

In [3]:
# Clear the vector datasbase 
if 0:
    vector_db.redo()

In [16]:
# Add the embedding vectors to the vector database
vector_db.populate(
    embedder=embedder,
    batch_size=10,
)

9791it [00:16, 595.09it/s]


## Set up the semantic search with chat interface

In [4]:
from search import SwedishLegislationSearch

In [6]:
law_search = SwedishLegislationSearch(
    embedd_function=EmbedderSwedishLegislation.embedd_single,
    vector_db=vector_db,
)

In [97]:
return_data = law_search.query("Vem har befogenheter att starta ett krig? Vem har ansvar att leda de väpnade styrkorna vid ett krig? Svara detaljerat", limit=5, print_debug=True)

###########CONTEXT###########

Förordning (2017:1249) om medinflytande för totalförsvarspliktiga|| respektive pluton, kompani, förband eller enhet ska föra de totalförsvarspliktigas talan gentemot chefen på samma ställe. 7 § Den chef som avses i 6 § ska regelbundet samla de totalförsvarspliktiga för information om och utvärdering av verksamheten. Vid dessa tillfällen ska chefen eller befattningshavare som denne har utsett vara ordförande och de totalförsvarspliktiga vara i majoritet. 8 § En del av den tid som avsatts för information och utvärdering enligt 7 § får användas av de totalförsvarspliktiga för egna överläggningar under ledning av den förtroendevalda. 9 § De förtroendevalda ska ges tillfälle att, efter överenskommelse med ansvarig chef, få samlas för förberedande sammanträde på tjänstetid. Medinflytande på central nivå 10 § För totalförsvarspliktiga som genomgår grundutbildning ska det årligen anordnas en kongress. Vid kongressen ska ombud för de totalförsvarspliktiga delta. D

In [72]:
# chat-gpt-3.5 reply
s = ''' 
Frågan om vem som har befogenhet att starta ett krig och vem som har ansvar att leda de väpnade styrkorna vid ett krig kan variera beroende på vilket land eller organisation vi talar om.

I allmänhet är det dock vanligt att beslutet om att starta ett krig fattas av statens högsta beslutsfattare, till exempel presidenten eller premiärministern, med stöd av regeringen och/eller parlamentet. I en del länder kan det även krävas att beslutet om krigsförklaringen godkänns av en domstol eller annan oberoende institution.

När det gäller vem som har ansvar att leda de väpnade styrkorna vid ett krig är det vanligtvis landets överbefälhavare eller högsta militära ledare. I vissa länder kan det också finnas en särskild minister eller byrå som ansvarar för försvar och militära frågor. Under själva kriget kan det finnas flera nivåer av militära befälhavare som ansvarar för olika aspekter av krigföringen, till exempel marktrupper, flyg eller sjöstridskrafter.
'''
print(s)

 
Frågan om vem som har befogenhet att starta ett krig och vem som har ansvar att leda de väpnade styrkorna vid ett krig kan variera beroende på vilket land eller organisation vi talar om.

I allmänhet är det dock vanligt att beslutet om att starta ett krig fattas av statens högsta beslutsfattare, till exempel presidenten eller premiärministern, med stöd av regeringen och/eller parlamentet. I en del länder kan det även krävas att beslutet om krigsförklaringen godkänns av en domstol eller annan oberoende institution.

När det gäller vem som har ansvar att leda de väpnade styrkorna vid ett krig är det vanligtvis landets överbefälhavare eller högsta militära ledare. I vissa länder kan det också finnas en särskild minister eller byrå som ansvarar för försvar och militära frågor. Under själva kriget kan det finnas flera nivåer av militära befälhavare som ansvarar för olika aspekter av krigföringen, till exempel marktrupper, flyg eller sjöstridskrafter.



In [81]:
return_data = law_search.query("Vem kan ej gifta sig med varandra? Svara detaljerat", limit=5, print_debug=True)

###########CONTEXT###########

Lag (1904:26 s.1) om vissa internationella rättsförhållanden rörande äktenskap och förmynderskap||1 kap. Om äktenskapets ingående 1 § Om två personer vill ingå äktenskap med varandra inför en svensk myndighet, ska det prövas om det finns något hinder mot äktenskapet enligt svensk lag. Om ingen av dem är svensk medborgare eller har hemvist i Sverige, ska det dessutom prövas att var och en av dem har rätt att ingå äktenskapet enligt lagen i en efter eget val angiven stat där han eller hon antingen är medborgare eller har hemvist. Om båda begär det och det finns särskilda skäl, får dock prövningen ske med tillämpning av enbart svensk lag även i ett sådant fall. Lag (2009:256). 2 § Har upphävts genom lag (2004:144). 3 § Har upphävts genom lag (2004:144). 4 § När äktenskap skall ingås inför svensk myndighet här i riket, tillämpas svensk lag i fråga om vigsel. Lag (1973:942). 5 § Regeringen kan bestämma, att med diplomatisk eller konsulär tjänst eller med befat

In [79]:
return_data = law_search.query("Vem kan bli medborgare? Svara detaljerat", limit=5, print_debug=True)

####CONTEXT####

Lag (2022:700) om särskild kontroll av vissa utlänningar|| avviker, håller sig undan eller på annat sätt hindrar verkställigheten eller utövar brottslig verksamhet i Sverige, eller 2. utlänningens identitet är oklar. 2 § En utlänning som inte har fyllt 18 år får tas i förvar, om det 1. är sannolikt att ett beslut om utvisning enligt 2 kap. 1 § kommer att meddelas, eller 2. finns ett beslut om utvisning enligt 2 kap. 1 § och förvar behövs för att förbereda eller genomföra verkställighet av beslutet. Utlänningen får dock tas i förvar endast om det finns en betydande risk för att utlänningen annars avviker, håller sig undan eller på annat sätt hindrar verkställigheten eller utövar brottslig verksamhet i Sverige. Vem som får besluta om förvar 3 § Ett beslut om förvar meddelas av den behöriga myndigheten, om inte annat följer av 4 §. Behörig myndighet är Migrationsverket, om inte något annat anges i tredje eller fjärde stycket. Regeringen är behörig myndighet från det att e

In [96]:
return_data = law_search.query("När blir det inte tillåtet att köpa en ny bensinbil? Svara detaljerat", limit=5, print_debug=True)

###########CONTEXT###########

Förordning (2022:206) om statligt stöd till försäljningsställen för drivmedel i vissa landsbygdsområden||Tidsbegränsad:2024-01-01
Förordning (2022:315) om miljökrav vid upphandling av bilar och vissa tjänster inom vägtransportområdet||varor med hög risk för indirekt ändring av markanvändning för vilka en betydande utvidgning av produktionsområdet till mark med stora kollager kan observeras och certifiering av biodrivmedel, flytande biobränslen och biomassabränslen med låg risk för indirekt ändrad markanvändning, eller - biodrivmedel, syntetiska bränslen eller paraffiniska bränslen som blandas med konventionella fossila bränslen.
Lag (2011:846) om miljökrav vid upphandling av bilar och vissa tjänster inom vägtransportområdet||Ikraft:2011-07-01 överg.best.
Förordning (2022:315) om miljökrav vid upphandling av bilar och vissa tjänster inom vägtransportområdet||Lagstödet för förordningen 1 § Denna förordning är meddelad med stöd av 12 § lagen (2011:846) om mi